### Comparison of the [PCR Primer Inspector](http://www.molbiotools.com/primerinspector.php) and the Biopython [Bio.SeqUtils.MeltingTemp](https://biopython.org/docs/1.74/api/Bio.SeqUtils.MeltingTemp.html) module

The objective of this analysis is to find out if PCR Primer Inspector and biopython calculate the same velue for Tm of primers. 

This comparison was done on November 5, 2019, Biopython version is 1.74

The example case is described below:

The PCR reaction is a version of this [standard](https://www.neb.com/protocols/0001/01/01/taq-dna-polymerase-with-standard-taq-buffer-m0273) protocol:

- 1X Taq Buffer with (NH4)2SO4
- 0.2 mM of each dNTP
- 2 mM MgCl2
- 1 µM Primer I 
- 1 µM Primer II 
- Template DNA

The buffer has this composition:

[10X Taq Buffer with (NH4)2SO4](https://www.thermofisher.com/order/catalog/product/B33?SID=srch-srp-B33#/B33?SID=srch-srp-B33) (B33) includes:
- 750 mM Tris-HCl (pH 8.8 at 25°C)
- 200 mM (NH4)2SO4
- 0.1% (v/v) Tween 20


This means that the final PCR reaction has: 

- 75 mM Tris-HCl
- 40 mM NH4+ 
- 20 mM SO42-
- 2  mM Mg2+
- 4  mM Cl-

The text below was copied from PCR Primer Inspector

    The total concentration of monovalent cations is expectedly a sum of concentrations of sodium, potassium and ammonium cations 
    (not all of them have to be present in a PCR reaction mix). It is also advised to add the concentration of the dissociated 
    fraction of Tris in the PCR reaction mix (a half of the total Tris concentration may be a good approximation). 
    The Tm prediction may be less accurate in high concentrations (above 100 mM) of monovalent ions capable to form hydrogen bonds 
    (ammonium, Tris) and for bulkier monovalent ions (tetramethylammonium).

Hence, "Cations" concentration should be 75/2 (half of Tris-HCl) + 40 (NH4+) = 77.5 mM

This value was added to the input form together with the other data:

![](page1.png)

The result is 54.4°C using the method of Owczarzy 2008

The text below was copied from PCR Primer Inspector

    MELTING TEMPERATURE CALCULATION

    All the melting temperature calculations provided here are based on the 
    nearest neighbor method with parameters described in references [1] and [4]. 

    Results of two different algorithms (references [2] and [3]) for monovalent 
    and magnesium cation correction are offered.


    1.  A unified view of polymer, dumbbell, and oligonucleotide DNA nearest-neighbor thermodynamics
        SantaLucia J. Jr. Proc Natl Acad Sci U S A. 1998;95(4):1460-5.

    2.  Oligonucleotide melting temperatures under PCR conditions: nearest-neighbor corrections for Mg(2+), 
        deoxynucleotide triphosphate, and dimethyl sulfoxide concentrations with comparison to alternative 
        empirical formulas
        von Ahsen N., Wittwer C. T., Schütz E.
        Clin Chem. 2001;47(11):1956-61.

    3.  Predicting stability of DNA duplexes in solutions containing magnesium and monovalent cations
        Owczarzy R., Moreira B. G., You Y., Behlke M. A., Walder J. A.
        Biochemistry. 2008;47(19):5336-53.

    4.  The thermodynamics of DNA structural motifs
        SantaLucia J. Jr., Hicks D.
        Annu Rev Biophys Biomol Struct. 2004;33:415-40.
        
        
From above can be deduced that:

1. parameters described in references [1] and [4]
2. The salt correction from Owczarzy 2008 is used.

The first statement issomewhat ambigous as the data in 1 and 4 is different (see below)
although versy similar.

#### Reference [1]
![](sl_and_hicks_2004_table1.png)

#### Reference [4]
![](sl_1998_table2.png)

### Biopython

From Biopython [docs](https://biopython.org/DIST/docs/api/Bio.SeqUtils.MeltingTemp-module.html):

table: Thermodynamic NN values, eight tables are implemented: For DNA/DNA hybridizations:

- DNA_NN1: values from Breslauer et al. (1986)
- DNA_NN2: values from Sugimoto et al. (1996)
- DNA_NN3: values from Allawi & SantaLucia (1997) (default)
- DNA_NN4: values from SantaLucia & Hicks (2004)                   <====<<<

I choose the DNA_NN4 as the SantaLucia 1998 data from PCR Primer Inspector does not seem to be available.
On closer inspection, Allawi & SantaLucia (1997) table 1 seem identical to SantaLucia J. Jr., Hicks D. 2004 Table 2.

The salt correction method from Owczarzy et al. (2008) was chosen since this was what was used for PCR Primer Inspector.

[7](https://biopython.org/DIST/docs/api/Bio.SeqUtils.MeltingTemp-module.html#salt_correction) = Complex formula with decision tree and 7 empirical constants. Mg2+ is corrected for dNTPs binding (if present) (Owczarzy et al. (2008), Biochemistry 47: 5336-5353)

In [46]:
import Bio
Bio.__version__

'1.74'

In [47]:
from Bio.SeqUtils import MeltingTemp as mt
print( mt.Tm_NN("ATGGCAGTTGAGAAGA", 
       nn_table=mt.DNA_NN4,
       Na=40,
       Tris=75.0,
       Mg=2.0, 
       dnac1=1000, 
       dnac2=0, 
       dNTPs=0.8, 
       saltcorr=7) )

57.794407437364214


The biopython module reports 57.79°C while the PCR Primer Inspector reported 54.4°C.
The difference is about 3.4 °C.